<a href="https://colab.research.google.com/github/kavyajeetbora/EAST/blob/master/EAST_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Imports

[**Tasks - Incidental Scene Text**](https://rrc.cvc.uab.es/?ch=4&com=tasks)

In [0]:
import os
from PIL import Image
from torch.utils.model_zoo import load_url
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn

In [275]:
%%time

!git clone -l -s git://github.com/kavyajeetbora/EAST.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 60 (delta 34), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (60/60), 388.08 KiB | 3.70 MiB/s, done.
Resolving deltas: 100% (34/34), done.
/content/cloned-repo/cloned-repo/cloned-repo/cloned-repo/cloned-repo/cloned-repo/cloned-repo/cloned-repo
dataset_original.py  EAST_dataset.ipynb        loss.py	 train.py
dataset.py	     EAST_text_detector.ipynb  model.py
CPU times: user 171 ms, sys: 225 ms, total: 396 ms
Wall time: 13.5 s


In [276]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [291]:
from dataset_original import custom_dataset

# AI4Bharat dataset
image_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/Simple YOLO model/Data/Sample Images'
label_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/Simple YOLO model/Data/Sample Annotations'


# icdar dataset
# image_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/EAST scene text detector/data/ICDAR dataset/Images'
# label_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/EAST scene text detector/data/ICDAR dataset/Labels'

training_dataset = custom_dataset(image_address, label_address)
print(len(training_dataset))

428


In [0]:
def extract_vertices(lines):
	'''extract vertices info from txt lines
	Input:
		lines   : list of string info
	Output:
		vertices: vertices of text regions <numpy.ndarray, (n,8)>
		labels  : 1->valid, 0->ignore, <numpy.ndarray, (n,)>
	'''
	labels = []
	vertices = []
	for line in lines:
		vertices.append(list(map(int,line.rstrip('\n').lstrip('\ufeff').split(',')[:8])))
		label = 0 if '##::' in line else 1
		labels.append(label)
	return np.array(vertices), np.array(labels)

In [0]:
from dataset_original import *

In [304]:
img = Image.open(image_address+'/0.jpg')

with open(label_address+'/0.txt', 'r') as f:
			lines = f.readlines()
vertices, labels = extract_vertices(lines)

img, vertices = adjust_height(img, vertices) 
img, vertices = rotate_img(img, vertices)
img, vertices = crop_img(img, vertices, labels, 512)

for vertice in vertices:
  print(vertice)
  print('-'*20)
  
print(labels)

[-139.9997009   295.70095097   57.41879004  126.65220186  -43.06110984
  215.63372402   34.31319314  274.43542797]
--------------------
[ 46.01064092 477.93392685 252.33446602 312.43005002 -36.46504421
 222.50208604  40.71021477 291.40708871]
--------------------
[237.76743583 596.26789713 371.14565557 488.84437564 -35.34358394
 222.52422429  45.30663585 285.88269799]
--------------------
[1 1 1]


In [305]:
score_map, geo_map, ignored_map = get_score_geo(img, vertices, labels, 0.25, 512)

torch.sum(score_map)

tensor(684.)

## Training

In [317]:
train_loader = DataLoader(training_dataset, shuffle = True, batch_size=4)

imgs, gt_score, gt_geo, ignored_maps = next(iter(train_loader))

print(imgs.size(), gt_score.size(), gt_geo.size(), ignored_maps.size())

print(torch.sum(gt_score))

torch.Size([4, 3, 512, 512]) torch.Size([4, 1, 128, 128]) torch.Size([4, 5, 128, 128]) torch.Size([4, 1, 128, 128])
tensor(3178.)


In [0]:
from model import EAST

model = EAST()

In [319]:
pred_score, pred_geo = model(imgs)

print(pred_score.size(), pred_geo.size())

torch.Size([4, 1, 128, 128]) torch.Size([4, 5, 128, 128])


In [320]:
from loss import Loss

loss_fn = Loss()

loss_fn(gt_score, pred_score, gt_geo, pred_geo, ignored_maps)

classify loss is 0.92153770, angle loss is 0.86955118, iou loss is 1.97817540


tensor(11.5952, grad_fn=<AddBackward0>)